In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from sklearn.linear_model import RidgeClassifier
import tensorflow as tf
import statsmodels.api as sm
import pickle

In [2]:
import sklearn

In [3]:
sklearn.__version__

'0.23.2'

# Reading in the Data

In [2]:
%%time
data = pd.read_csv('data.csv')

<decorator-gen-54>:2: DtypeWarning: Columns (42,166,167,168,169,174,175,178,179,182,183,188,189,190,191,194,195,203,204,205,218,219,220,231,232,233,238,240,241,249) have mixed types.Specify dtype option on import or set low_memory=False.


CPU times: user 33.3 s, sys: 7.76 s, total: 41 s
Wall time: 44 s


In [3]:
data.to_pickle('data.pickle')

In [4]:
data['game_date'] = pd.to_datetime(data['game_date'])

# Creating a target -> Win or not

In [5]:
data = data[data['score_differential'].isna() == False] # Dropping NaN values from score_differential

In [6]:
lookup_wins = data.groupby('game_id').tail(1)
lookup_wins = lookup_wins.set_index("game_id")
lookup_wins["wonGame"] = lookup_wins["score_differential"] > 0

In [7]:
lookup_wins = lookup_wins[["wonGame"]]

In [8]:
data = data.merge(lookup_wins, right_on=["game_id"], left_on=["game_id"])

Home Field Advantage %

In [9]:
lookup_wins["wonGame"].value_counts()[True] / lookup_wins["wonGame"].value_counts().sum()

0.6064924782264449

In [10]:
data["game_seconds_remaining"]

0         3593.0
1         3556.0
2         3515.0
3         3507.0
4         3496.0
           ...  
433947      64.0
433948      63.0
433949      58.0
433950      38.0
433951      35.0
Name: game_seconds_remaining, Length: 433952, dtype: float64

# Relevant Data

In [17]:
relevant_data = data[["score_differential", "game_seconds_remaining", "wonGame"]]


In [18]:
# relevant_data = relevant_data[relevant_data["game_seconds_remaining"]

In [19]:
relevant_data.dropna(inplace=True)

<ipython-input-19-5933214d0cd8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  relevant_data.dropna(inplace=True)


In [20]:
X_train, X_test, y_train, y_test = train_test_split(relevant_data.drop(columns=["wonGame"]), relevant_data["wonGame"], test_size=0.2, random_state=42, stratify=relevant_data["wonGame"])

In [21]:
train, test = train_test_split(relevant_data, test_size=0.2, random_state=42, stratify=relevant_data["wonGame"])

# Modeling

In [22]:
%%time
model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
model.fit(X_train, y_train)
model.score(X_test, y_test)

CPU times: user 34.8 s, sys: 823 ms, total: 35.6 s
Wall time: 35.9 s


0.6153181388703247

In [23]:
results = model.predict(X_test)

In [24]:
model.predict_proba([[-40, 30]])

array([[0.90990739, 0.09009261]])

In [29]:
with open('./modelDeploy/model.pkl','wb') as f:
    pickle.dump(model,f)
